# Developing a module for EMvidence

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from emvincelib import iq, ml, stat
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from scipy.fftpack import fft
from sklearn import preprocessing

from joblib import dump, load
import importlib.util

%matplotlib inline

### Training and Testing a Machine Learning Model

With these insights we have gained during the previous step, now we can generate the **X** and **y** training dataset for a classifier. We do that by sending a sliding window over the data files and converting each window data segment into a feature vector. Each feature vector is appended to the **X** matrix along with the appropriate label for it in the **y** vector.

In [ ]:
iq.sampleRate = 32e3
sliding_window = 0.1
feature_vector_size = 50
file1="./data/class-1.npy"
file2="./data/class-2.npy"

duration1 = iq.getTimeDuration(file1, fileType="npy")
duration2 = iq.getTimeDuration(file2, fileType="npy")

ml.loadTrainingData(file1, iq.sampleRate, feature_vector_size, sliding_window, duration1, "Class 1")
ml.loadTrainingData(file2, iq.sampleRate, feature_vector_size, sliding_window, duration2, "Class 2")

clf = ml.createClassifier()
ml.trainAndTest(clf)

### Saving the trained model to a file

In [ ]:
module_name = "binary-classifier"


dump(clf, "./modules/" + module_name + "/ml-model.joblib") 

### Packaging the files to a module

Create a new folder with a suitable name for the module. I'll name it **binary-classifier**. Then, we need 4 files in it.

- README.txt
- config.config
- ml-model.joblib
- main.py

#### README.txt

This file should contain the basic details of the module and how it should be used.

#### config.config

This file contains the required configurations for the module.

#### ml-model.joblib

This is the trained model that we saved previously.

#### main.py

Module can contain one or more Python files. However, it has to contain at least this **main.py** file.

#### Completing the module

Now, copy those 4 files into a folder named as **binary-classifier** and create a ZIP file. You can now import this into EMvidence as a module.

### Testing the Module

In [ ]:
def loadModule(module_path):
    spec = importlib.util.spec_from_file_location("main", module_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

module_path = "./modules/" + str(module_name) + "/" + "main.py"
# loading module
mod = loadModule(module_path)
# calling module functions
mod.initialize(1, "./data/class-1.npy", "./results")
results = mod.getResults()
print("Module results: " + str(results))